<a href="https://colab.research.google.com/github/aymanboufarhi/Arabic-Text-Preprocessing/blob/main/Arabic_Text_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab** : Get familiar with Scraping and NLP Pipeline ( Arabic Text )

Import Libraries :

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import pymongo
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.stem.isri import ISRIStemmer
from nltk.stem.arlstem import ARLSTem
from nltk import ne_chunk
from ar_corrector.corrector import Corrector

# **Data Acquisition**

Extracting raw content from the website mawdoo3 :

In [ ]:
url='https://mawdoo3.com/%D8%AB%D9%82%D8%A7%D9%81%D8%A9_%D8%B9%D8%A7%D9%85%D8%A9_%D8%AD%D9%88%D9%84_%D8%A7%D9%84%D9%85%D8%BA%D8%B1%D8%A8'
req=requests.get(url)
content=req.text

Parse the raw content into a readable format :

In [ ]:
soup=BeautifulSoup(content, "html.parser")

Connection to MongoDB Atlas :

In [ ]:
# Define the MongoDB Atlas connection string
connection_string = ""

# Connect to MongoDB Atlas
client = pymongo.MongoClient(connection_string)

# Access the database and collection
db = client["web_scraping_db"]
collection = db["web_data_collection"]

Store the raw data into a MongoDB collection :

In [ ]:
# Store the entire HTML content
data_to_store = {
    "url": url,
    "html_content": str(soup)
}

# Insert the data into MongoDB
insert_result = collection.insert_one(data_to_store)
print("Data inserted with ID:", insert_result.inserted_id)

Data inserted with ID: 66113baac154ef6dd4e56c11


Retrieve data from MongoDB :

In [ ]:
collection = db["web_data_collection"]

data_from_db = collection.find_one()

# Extract HTML content from data
html_content = data_from_db["html_content"]

# Parsing HTML content using BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Extract text from HTML
text = soup.get_text()

# **Text Cleaning**

Function to clean the raw data :

In [ ]:
def clean_text(txt):
    # Remove HTML tags and non-Arabic characters
    clean = re.sub('<[^<]+?>', '', txt)
    # Define regex pattern to match Arabic alphabet and spaces
    pattern = re.compile(r'[^\u0600-\u06FF\s]+')
    # Substitute non-matching characters with an empty string
    clean = re.sub(pattern, '', clean)
    # Remove extra spaces
    clean = re.sub(r'\s+', ' ', clean)
    return clean

def extract_and_clean_text(soup):
    # Find all h1 and p tags
    tags = soup.find_all(['h1', 'p'])
    # Extract text from tags
    extracted_text = ''
    for tag in tags:
        extracted_text += tag.get_text() + '\n'
    # Clean extracted text
    cleaned_text = clean_text(extracted_text)
    return cleaned_text


# Print the cleaned text
cleaned_text = extract_and_clean_text(soup)

cleaned_text

'ثقافة عامة حول المغرب تمت الكتابة بواسطة محمد مروان آخر تحديث ١٨٣١ ، ٣ فبراير ٢٠٢١ تقع المغرب في شمال قارة أفريقي، ويتحدّد موقعها جغرافياً بإحداثيات درجة شمالاً و درجات غرباً، إذ إنّ لها سواحل على المحيط الأطلسي والبحر الأبيض المتوسط الذي يفصل بينهما مضيق جبل طارق، كما تحدُّها من الجانب الشرقي دولة الجزائر، ومن الجانب الجنوبيّ الجمهورية الموريتانية، ويُشار إلى أنّ مساحة المغرب الإجمالية تبلغ حوالي كم١٢ تُعدّ مدينة الرباط عاصمة المغرب، ويُقيم فيها حوالي نسمة، وفقاً لإحصائيات شهر تموز من عام م، حيث يتوزّعون على مساحة تبلغ كم، إذ تبلغ الكثافة السكانية فيها نحو شخص لكلّ كيلومتر مربع، ويُطلق على مدينة الرباط عدّة ألقاب، مثل واشنطن شمال أفريقيا؛ لأنّ شوارعها واسعة ومعالمها الأثرية مميزة، بالإضافة إلى مبانيها الحكومية والمنازل التي تمّ تحويلها إلى فنادق، وتُسمّى أيضاً بمدينة الزهور لاحتوائها على حدائق شاسعة وجميلة، كما تُلقّب الرباط بالمدينة البيضاء نظراً إلى بيئتها الهادئة٣ اعتمدت المغرب العلم الوطنيّ رسمياً في السابع عشر من شهر تشرين الثاني لعام م، حيث كان قبل ذلك رايةً باللون الأحمر فقط د



---


Correct word spelling using the context ( in our case we don't need it, because we extract the text from a professional website ) :

In [ ]:
corr = Corrector()

correct_text = corr.contextual_correct(cleaned_text)

# **Text Preprocessing**

## Unicode Normalization

In [ ]:
normalized_text = cleaned_text.encode('UTF-16')

## Tokenization

In [ ]:
nltk.download('punkt')

In [ ]:
normalized_text_str = normalized_text.decode('utf-16')
tokens = word_tokenize(normalized_text_str)
print("Tokens:", tokens)

Tokens: ['ثقافة', 'عامة', 'حول', 'المغرب', 'تمت', 'الكتابة', 'بواسطة', 'محمد', 'مروان', 'آخر', 'تحديث', '١٨٣١', '،', '٣', 'فبراير', '٢٠٢١', 'تقع', 'المغرب', 'في', 'شمال', 'قارة', 'أفريقي،', 'ويتحدّد', 'موقعها', 'جغرافياً', 'بإحداثيات', 'درجة', 'شمالاً', 'و', 'درجات', 'غرباً،', 'إذ', 'إنّ', 'لها', 'سواحل', 'على', 'المحيط', 'الأطلسي', 'والبحر', 'الأبيض', 'المتوسط', 'الذي', 'يفصل', 'بينهما', 'مضيق', 'جبل', 'طارق،', 'كما', 'تحدُّها', 'من', 'الجانب', 'الشرقي', 'دولة', 'الجزائر،', 'ومن', 'الجانب', 'الجنوبيّ', 'الجمهورية', 'الموريتانية،', 'ويُشار', 'إلى', 'أنّ', 'مساحة', 'المغرب', 'الإجمالية', 'تبلغ', 'حوالي', 'كم١٢', 'تُعدّ', 'مدينة', 'الرباط', 'عاصمة', 'المغرب،', 'ويُقيم', 'فيها', 'حوالي', 'نسمة،', 'وفقاً', 'لإحصائيات', 'شهر', 'تموز', 'من', 'عام', 'م،', 'حيث', 'يتوزّعون', 'على', 'مساحة', 'تبلغ', 'كم،', 'إذ', 'تبلغ', 'الكثافة', 'السكانية', 'فيها', 'نحو', 'شخص', 'لكلّ', 'كيلومتر', 'مربع،', 'ويُطلق', 'على', 'مدينة', 'الرباط', 'عدّة', 'ألقاب،', 'مثل', 'واشنطن', 'شمال', 'أفريقيا؛', 'لأنّ', 'شوار

## Stop words

In [ ]:
nltk.download('stopwords')

All stop words that exist in arabic :

In [ ]:
# Get Arabic stop words from NLTK corpus
arabic_stopwords = nltk.corpus.stopwords.words('arabic')

print(arabic_stopwords)

['إذ', 'إذا', 'إذما', 'إذن', 'أف', 'أقل', 'أكثر', 'ألا', 'إلا', 'التي', 'الذي', 'الذين', 'اللاتي', 'اللائي', 'اللتان', 'اللتيا', 'اللتين', 'اللذان', 'اللذين', 'اللواتي', 'إلى', 'إليك', 'إليكم', 'إليكما', 'إليكن', 'أم', 'أما', 'أما', 'إما', 'أن', 'إن', 'إنا', 'أنا', 'أنت', 'أنتم', 'أنتما', 'أنتن', 'إنما', 'إنه', 'أنى', 'أنى', 'آه', 'آها', 'أو', 'أولاء', 'أولئك', 'أوه', 'آي', 'أي', 'أيها', 'إي', 'أين', 'أين', 'أينما', 'إيه', 'بخ', 'بس', 'بعد', 'بعض', 'بك', 'بكم', 'بكم', 'بكما', 'بكن', 'بل', 'بلى', 'بما', 'بماذا', 'بمن', 'بنا', 'به', 'بها', 'بهم', 'بهما', 'بهن', 'بي', 'بين', 'بيد', 'تلك', 'تلكم', 'تلكما', 'ته', 'تي', 'تين', 'تينك', 'ثم', 'ثمة', 'حاشا', 'حبذا', 'حتى', 'حيث', 'حيثما', 'حين', 'خلا', 'دون', 'ذا', 'ذات', 'ذاك', 'ذان', 'ذانك', 'ذلك', 'ذلكم', 'ذلكما', 'ذلكن', 'ذه', 'ذو', 'ذوا', 'ذواتا', 'ذواتي', 'ذي', 'ذين', 'ذينك', 'ريث', 'سوف', 'سوى', 'شتان', 'عدا', 'عسى', 'عل', 'على', 'عليك', 'عليه', 'عما', 'عن', 'عند', 'غير', 'فإذا', 'فإن', 'فلا', 'فمن', 'في', 'فيم', 'فيما', 'فيه', 'فيها', '

Filtering the tokens :

In [ ]:
# Filter stop words from Arabic tokens
filtered_arabic_tokens = [w for w in tokens if not w in arabic_stopwords]

print(filtered_arabic_tokens)

['ثقافة', 'حول', 'المغرب', 'تمت', 'الكتابة', 'بواسطة', 'محمد', 'مروان', 'آخر', 'تحديث', '١٨٣١', '،', '٣', '٢٠٢١', 'تقع', 'المغرب', 'قارة', 'أفريقي،', 'ويتحدّد', 'موقعها', 'جغرافياً', 'بإحداثيات', 'درجة', 'شمالاً', 'درجات', 'غرباً،', 'إنّ', 'سواحل', 'المحيط', 'الأطلسي', 'والبحر', 'الأبيض', 'المتوسط', 'يفصل', 'بينهما', 'مضيق', 'جبل', 'طارق،', 'تحدُّها', 'الجانب', 'الشرقي', 'دولة', 'الجزائر،', 'الجانب', 'الجنوبيّ', 'الجمهورية', 'الموريتانية،', 'ويُشار', 'أنّ', 'مساحة', 'المغرب', 'الإجمالية', 'تبلغ', 'حوالي', 'كم١٢', 'تُعدّ', 'مدينة', 'الرباط', 'عاصمة', 'المغرب،', 'ويُقيم', 'حوالي', 'نسمة،', 'وفقاً', 'لإحصائيات', 'شهر', 'عام', 'م،', 'يتوزّعون', 'مساحة', 'تبلغ', 'كم،', 'تبلغ', 'الكثافة', 'السكانية', 'شخص', 'لكلّ', 'كيلومتر', 'مربع،', 'ويُطلق', 'مدينة', 'الرباط', 'عدّة', 'ألقاب،', 'واشنطن', 'أفريقيا؛', 'لأنّ', 'شوارعها', 'واسعة', 'ومعالمها', 'الأثرية', 'مميزة،', 'بالإضافة', 'مبانيها', 'الحكومية', 'والمنازل', 'تمّ', 'تحويلها', 'فنادق،', 'وتُسمّى', 'أيضاً', 'بمدينة', 'الزهور', 'لاحتوائها', 'حد

## **Stemming** VS **Lemmatization**

---



## Stemming :

**ISRIStemmer** :
The ISRIStemmer is a stemming algorithm specifically designed for Arabic text. It applies a set of rules to reduce words to their base forms. While ISRIStemmer can effectively reduce words to their stems, it may occasionally produce stems that do not correspond to valid words in the Arabic language. This is because the algorithm focuses on morphological patterns rather than semantic meaning.

In [ ]:
# Initialize ISRIStemmer
stemmer = ISRIStemmer()

# Stem the filtered Arabic tokens
stemmed_tokens = [stemmer.stem(w) for w in filtered_arabic_tokens]

print("Filtered Arabic Tokens (After Stop Words Removal and Stemming):")
print(stemmed_tokens)

Filtered Arabic Tokens (After Stop Words Removal and Stemming):
['ثقف', 'حول', 'غرب', 'تمت', 'كتب', 'بسط', 'حمد', 'مرو', 'اخر', 'حدث', '١٨٣١', '،', '٣', '٢٠٢١', 'تقع', 'غرب', 'قرة', 'ريقي،', 'حدد', 'وقع', 'جغراف', 'إحداث', 'درج', 'شمل', 'درج', 'غربا،', 'ان', 'سحل', 'حيط', 'طلس', 'بحر', 'ابض', 'توسط', 'فصل', 'بين', 'ضيق', 'جبل', 'طارق،', 'تحد', 'جنب', 'شرق', 'دول', 'جزائر،', 'جنب', 'جنب', 'جمهور', 'موريتانية،', 'يشر', 'ان', 'ساح', 'غرب', 'جمل', 'بلغ', 'حلي', 'كم١٢', 'تعد', 'دين', 'ربط', 'عصم', 'غرب،', 'يقم', 'حلي', 'مة،', 'وفق', 'إحصائ', 'شهر', 'عام', 'م،', 'وزع', 'ساح', 'بلغ', 'كم،', 'بلغ', 'كثف', 'سكن', 'شخص', 'لكل', 'كيلومتر', 'ربع،', 'طلق', 'دين', 'ربط', 'عدة', 'قب،', 'شنط', 'افريقيا؛', 'لأن', 'شرع', 'وسع', 'علم', 'ثري', 'مميزة،', 'ضفة', 'بني', 'حكم', 'نزل', 'تم', 'حول', 'دق،', 'سمى', 'ايض', 'بمد', 'زهر', 'حوئ', 'حدق', 'شسع', 'جميلة،', 'لقب', 'ربط', 'دين', 'يضء', 'نظر', 'بيئ', 'هادئة٣', 'عمد', 'غرب', 'علم', 'وطن', 'رسم', 'سبع', 'شهر', 'ثني', 'لعم', 'م،', 'رية', 'لون', 'حمر', 'فقط', 

**SnowballStemmer** **for** **Arabic** :
SnowballStemmer is a popular stemming algorithm available for various languages, including Arabic. It uses a set of rules and heuristics to truncate words to their root forms. Similar to ISRIStemmer, SnowballStemmer may generate stems that are not valid Arabic words due to its rule-based approach.

In [ ]:
nltk.download('snowball_data')

# Initialize SnowballStemmer for Arabic
stemmer = SnowballStemmer("arabic")

In [ ]:
# Stem Arabic tokens
stemmed_tokens = [stemmer.stem(token) for token in filtered_arabic_tokens]

print("Filtered Arabic Tokens (After Stop Words Removal and Stemming):")
print(stemmed_tokens)

Filtered Arabic Tokens (After Stop Words Removal and Stemming):
['ثقاف', 'حول', 'مغرب', 'تمت', 'كتاب', 'واسط', 'محمد', 'مروا', 'اخر', 'تحديث', '١٨٣١', '', '٣', '٢٠٢١', 'تقع', 'مغرب', 'قار', 'افريق', 'يتحدد', 'موقع', 'جغراف', 'احداث', 'درج', 'شمال', 'درج', 'غرب', 'إن', 'سواحل', 'محيط', 'اطلس', 'بحر', 'ابيض', 'متوسط', 'يفصل', 'بين', 'مضيق', 'جبل', 'طارق', 'تحد', 'جانب', 'شرق', 'دول', 'جزاير', 'جانب', 'جنوب', 'جمهور', 'موريتان', 'يشار', 'أن', 'مساح', 'مغرب', 'اجمال', 'تبلغ', 'حوال', 'كم١٢', 'تعد', 'مدين', 'رباط', 'عاصم', 'مغرب', 'يقيم', 'حوال', 'نسم', 'وفق', 'احصاء', 'شهر', 'عام', 'م', 'يتوزع', 'مساح', 'تبلغ', 'كم', 'تبلغ', 'كثاف', 'سكان', 'شخص', 'لكل', 'يلومتر', 'مربع', 'يطلق', 'مدين', 'رباط', 'عد', 'القاب', 'واشنط', 'افريق', 'لان', 'شوارع', 'واسع', 'معالم', 'اثر', 'مميز', 'اضاف', 'مبان', 'حكوم', 'منازل', 'تم', 'تحويل', 'نادق', 'تسمى', 'ايض', 'مدين', 'زهور', 'احتواء', 'حدايق', 'شاسع', 'جميل', 'تلقب', 'رباط', 'مدين', 'بيضاء', 'نظر', 'بيء', 'هادية٣', 'اعتمد', 'مغرب', 'علم', 'وطن', 'رسم', '

**ARLSTem** :
ARLSTem is another stemming algorithm for Arabic text processing. It aims to improve upon the limitations of ISRIStemmer and SnowballStemmer by considering linguistic and semantic factors in addition to morphological patterns. However, like other stemming methods, it may still produce stems that deviate from actual dictionary entries.

In [ ]:
# Initialize ARLSTem
stemmmer = ARLSTem()

# Stem Arabic tokens
stemmed_tokens = [stemmer.stem(token) for token in filtered_arabic_tokens]

print("Filtered Arabic Tokens (After Stop Words Removal and Stemming):")
print(stemmed_tokens)

Filtered Arabic Tokens (After Stop Words Removal and Stemming):
['ثقاف', 'حول', 'مغرب', 'تمت', 'كتاب', 'واسط', 'محمد', 'مروا', 'اخر', 'تحديث', '١٨٣١', '', '٣', '٢٠٢١', 'تقع', 'مغرب', 'قار', 'افريق', 'يتحدد', 'موقع', 'جغراف', 'احداث', 'درج', 'شمال', 'درج', 'غرب', 'إن', 'سواحل', 'محيط', 'اطلس', 'بحر', 'ابيض', 'متوسط', 'يفصل', 'بين', 'مضيق', 'جبل', 'طارق', 'تحد', 'جانب', 'شرق', 'دول', 'جزاير', 'جانب', 'جنوب', 'جمهور', 'موريتان', 'يشار', 'أن', 'مساح', 'مغرب', 'اجمال', 'تبلغ', 'حوال', 'كم١٢', 'تعد', 'مدين', 'رباط', 'عاصم', 'مغرب', 'يقيم', 'حوال', 'نسم', 'وفق', 'احصاء', 'شهر', 'عام', 'م', 'يتوزع', 'مساح', 'تبلغ', 'كم', 'تبلغ', 'كثاف', 'سكان', 'شخص', 'لكل', 'يلومتر', 'مربع', 'يطلق', 'مدين', 'رباط', 'عد', 'القاب', 'واشنط', 'افريق', 'لان', 'شوارع', 'واسع', 'معالم', 'اثر', 'مميز', 'اضاف', 'مبان', 'حكوم', 'منازل', 'تم', 'تحويل', 'نادق', 'تسمى', 'ايض', 'مدين', 'زهور', 'احتواء', 'حدايق', 'شاسع', 'جميل', 'تلقب', 'رباط', 'مدين', 'بيضاء', 'نظر', 'بيء', 'هادية٣', 'اعتمد', 'مغرب', 'علم', 'وطن', 'رسم', '

## Lemmatization :

**Farasa API** :
Farasa is an Arabic NLP toolkit that provides lemmatization functionality through its web API. It leverages machine learning models and linguistic resources to accurately map words to their base forms. Unlike stemming, lemmatization considers the context and meaning of words, resulting in more accurate normalization. However, it may introduce computational overhead due to its reliance on external resources.

In [ ]:
# Define the URL of the Farasa lemmatization API
url = 'https://farasa.qcri.org/webapi/lemmatization/'

# API key provided by Farasa
api_key = "EkhjeVroQrUHSKjdfI"

# Prepare the payload data
payload = {'text': ' '.join(filtered_arabic_tokens), 'api_key': api_key}

# Send a POST request to the Farasa lemmatization API
response = requests.post(url, data=payload)

# Parse the JSON response
result = json.loads(response.text)

# Extract the list of tokens
tokens = result.get('text', [])

# Remove '،' from each token
tokens_lemma = [token.replace('،', '') for token in tokens]

# Filter out empty tokens
tokens_lemma = [token for token in tokens_lemma if token]

# Print the list of tokens
print(tokens_lemma)

['ثقافة', 'حول', 'مغرب', 'تم', 'كتابة', 'واسطة', 'محمد', 'مروان', 'آخر', 'تحديث', '1831', '3', '2021', 'وقع', 'مغرب', 'قارة', 'أفريقي', 'تحدد', 'موقع', 'جغرافي', 'إحداثي', 'درجة', 'شمال', 'درجة', 'غرب', 'إن', 'ساحل', 'محيط', 'أطلسي', 'بحر', 'أبيض', 'متوسط', 'فصل', 'بين', 'مضيق', 'جبل', 'طارق', 'حد', 'جانب', 'شرقي', 'دولة', 'جزائر', 'جانب', 'جنوبي', 'جمهورية', 'موريتاني', 'أشار', 'أن', 'مساحة', 'مغرب', 'إجمالي', 'بلغ', 'حوالي', 'كم', '12', 'عد', 'مدينة', 'رباط', 'عاصمة', 'مغرب', 'أقام', 'حوالي', 'نسمة', 'وفق', 'إحصائي', 'شهر', 'عام', 'م', 'توزع', 'مساحة', 'بلغ', 'كم', 'بلغ', 'كثافة', 'سكاني', 'شخص', 'كل', 'كيلومتر', 'مربع', 'أطلق', 'مدينة', 'رباط', 'عدة', 'لقب', 'واشنطون', 'أفريقيا', '؛', 'أن', 'شارع', 'واسع', 'معلم', 'أثري', 'مميز', 'إضافة', 'مبنى', 'حكومي', 'منزل', 'تم', 'تحويل', 'فندق', 'سمى', 'أيض', 'مدينة', 'زهور', 'احتواء', 'حديقة', 'شاسع', 'جميل', 'تلقب', 'رباط', 'مدينة', 'أبيض', 'نظر', 'بيئة', 'هادئ', '3', 'اعتمد', 'مغرب', 'علم', 'وطني', 'رسمي', 'سابع', 'شهر', 'ثاني', 'عام', 'م'

**Comparison** :

*   Stemming methods such as ISRIStemmer, SnowballStemmer, and ARLSTem offer fast and lightweight normalization of Arabic text but may produce stems that are not valid words. For example, when stemming the word 'والبرتغال' (meaning 'and Portugal'), the Farasa API yields 'برتغال', which is a valid Arabic word, while the other stemmers provide 'رتغال', which may not be recognized as a valid word. Similarly, for the word 'إسبانيا' (meaning 'Spain'), the stemmers output 'سبن', whereas the Farasa API retains the original word, 'إسبانيا'. These examples highlight instances where the Farasa API preserves the semantic integrity of the words while the stemmers prioritize morphological reduction, sometimes resulting in stems that do not correspond to valid entries in the Arabic language.

*   Lemmatization with Farasa API provides more accurate normalization by considering semantics and context, but it may be slower and require internet connectivity.

In summary, the choice between stemming and lemmatization depends on the specific requirements of the NLP task. While stemming is suitable for general text processing tasks that prioritize speed and simplicity, lemmatization is preferred for applications where accuracy and semantic fidelity are paramount.





---


## Parts of Speech (POS) :

Rule Based Approach :

In [ ]:
# Define rules for part-of-speech tagging using regular expressions
patterns = [
    (r'^ال.*$', 'اسم'),            # Nouns starting with ال
    (r'^و.*$', 'اسم'),              # Nouns starting with و
    (r'^ب.*$', 'اسم'),              # Nouns starting with ب
    (r'^ف.*$', 'اسم'),              # Nouns starting with ف
    (r'^ك.*$', 'اسم'),              # Nouns starting with ك
    (r'^ل.*$', 'اسم'),              # Nouns starting with ل
    (r'^م.*$', 'اسم'),              # Nouns starting with م
    (r'^ن.*$', 'اسم'),              # Nouns starting with ن
    (r'^ت.*$', 'اسم'),              # Nouns starting with ت
    (r'^ه.*$', 'اسم'),              # Nouns starting with ه
    (r'^ي.*$', 'اسم'),              # Nouns starting with ي
    (r'^ع.*$', 'اسم'),              # Nouns starting with ع
    (r'^ص.*$', 'اسم'),              # Nouns starting with ص
    (r'^ح.*$', 'اسم'),              # Nouns starting with ح
    (r'^ق.*$', 'اسم'),              # Nouns starting with ق
    (r'^ر.*$', 'اسم'),              # Nouns starting with ر
    (r'^س.*$', 'اسم'),              # Nouns starting with س
    (r'^ش.*$', 'اسم'),              # Nouns starting with ش
    (r'^ض.*$', 'اسم'),              # Nouns starting with ض
    (r'^ظ.*$', 'اسم'),              # Nouns starting with ظ
    (r'^غ.*$', 'اسم'),              # Nouns starting with غ
    (r'^هذا$', 'اسم'),             # Specific words like هذا
    (r'^هذه$', 'اسم'),             # Specific words like هذه
    (r'^هو$', 'ضمير'),              # Specific pronouns like هو
    (r'^هي$', 'ضمير'),              # Specific pronouns like هي
    (r'.*ُ$', 'اسم'),               # Nouns ending with ُ
    (r'.*َة$', 'اسم'),              # Nouns ending with َة
    (r'.*ِ$', 'فعل'),               # Verbs ending with ِ
    (r'.*َ$', 'صفة'),               # Adjectives ending with َ
    (r'.*َةُ$', 'اسم'),            # Nouns ending with َةُ
    (r'.*َةِ$', 'اسم'),            # Nouns ending with َةِ
    (r'.*ُون$', 'اسم'),            # Nouns ending with ُون
    (r'.*ِين$', 'اسم'),            # Nouns ending with ِين
    (r'.*ْ$', 'اسم'),              # Nouns ending with ْ
    (r'.*ى$', 'ضمير'),              # Pronouns ending with ى
    (r'.*َه$', 'ضمير'),             # Pronouns ending with ه
    (r'.*ت$', 'فعل'),              # Verbs ending with ت
    (r'.*نا$', 'ضمير'),             # Pronouns ending with نا
    (r'.*ك$', 'ضمير'),              # Pronouns ending with ك
    (r'.*ي$', 'ضمير'),              # Pronouns ending with ي
    (r'.*ُو$', 'ضمير'),             # Pronouns ending with ُو
    (r'.*ها$', 'ضمير'),             # Pronouns ending with ها
    (r'.*كما$', 'ضمير'),            # Pronouns ending with كما
    (r'.*كن$', 'ضمير'),             # Pronouns ending with كن
    (r'.*كم$', 'ضمير'),             # Pronouns ending with كم
    (r'.*ني$', 'ضمير'),             # Pronouns ending with ني
    (r'.*هما$', 'ضمير'),            # Pronouns ending with هما
    (r'.*هم$', 'ضمير'),             # Pronouns ending with هم
    (r'.*هن$', 'ضمير'),             # Pronouns ending with هن
    (r'.*ه$', 'ضمير'),              # Pronouns ending with ه
    (r'.*تا$', 'فعل'),              # Verbs ending with تا
    (r'.*تما$', 'فعل'),             # Verbs ending with تما
    (r'.*وا$', 'فعل'),              # Verbs ending with وا
    (r'.*ن$', 'فعل'),               # Verbs ending with ن
    (r'.*ه$', 'فعل'),               # Verbs ending with ه
    (r'.*و$', 'فعل'),               # Verbs ending with و
    (r'.*ي$', 'فعل'),               # Verbs ending with ي
    (r'^\w+$', 'اسم')               # Default to NOUN if no pattern matches
]

# Compile the patterns into a tagger
regexp_tagger = nltk.RegexpTagger(patterns)

# Tag the words using the rule-based tagger
tagged_words = regexp_tagger.tag(filtered_arabic_tokens)

# Print the tagged words
print(tagged_words)

[('ثقافة', 'اسم'), ('حول', 'اسم'), ('المغرب', 'اسم'), ('تمت', 'اسم'), ('الكتابة', 'اسم'), ('بواسطة', 'اسم'), ('محمد', 'اسم'), ('مروان', 'اسم'), ('آخر', 'اسم'), ('تحديث', 'اسم'), ('١٨٣١', 'اسم'), ('،', None), ('٣', 'اسم'), ('٢٠٢١', 'اسم'), ('تقع', 'اسم'), ('المغرب', 'اسم'), ('قارة', 'اسم'), ('أفريقي،', None), ('ويتحدّد', 'اسم'), ('موقعها', 'اسم'), ('جغرافياً', None), ('بإحداثيات', 'اسم'), ('درجة', 'اسم'), ('شمالاً', 'اسم'), ('درجات', 'فعل'), ('غرباً،', 'اسم'), ('إنّ', None), ('سواحل', 'اسم'), ('المحيط', 'اسم'), ('الأطلسي', 'اسم'), ('والبحر', 'اسم'), ('الأبيض', 'اسم'), ('المتوسط', 'اسم'), ('يفصل', 'اسم'), ('بينهما', 'اسم'), ('مضيق', 'اسم'), ('جبل', 'اسم'), ('طارق،', None), ('تحدُّها', 'اسم'), ('الجانب', 'اسم'), ('الشرقي', 'اسم'), ('دولة', 'اسم'), ('الجزائر،', 'اسم'), ('الجانب', 'اسم'), ('الجنوبيّ', 'اسم'), ('الجمهورية', 'اسم'), ('الموريتانية،', 'اسم'), ('ويُشار', 'اسم'), ('أنّ', None), ('مساحة', 'اسم'), ('المغرب', 'اسم'), ('الإجمالية', 'اسم'), ('تبلغ', 'اسم'), ('حوالي', 'اسم'), ('كم١٢', 

Machine Learning Approach :

the NLTK library provides both rule-based and machine learning-based approaches for natural language processing tasks. However, the part-of-speech tagging method we used with NLTK (nltk.pos_tag) utilizes a pre-trained statistical model based on a machine learning algorithm. This algorithm has been trained on annotated corpora to learn patterns in the data and predict part-of-speech tags for unseen text based on these learned patterns. Therefore, it falls under the category of machine learning-based approach.

In [ ]:
nltk.download('averaged_perceptron_tagger')

In [ ]:
# Perform part-of-speech tagging
tagged_tokens = nltk.pos_tag(stemmed_tokens)
print(tagged_tokens)

[('ثقف', 'JJ'), ('حول', 'NNP'), ('غرب', 'NNP'), ('تمت', 'NNP'), ('كتب', 'NNP'), ('بسط', 'NNP'), ('حمد', 'NNP'), ('مرو', 'NNP'), ('اخر', 'NNP'), ('حدث', 'NNP'), ('١٨٣١', 'CD'), ('،', 'VBD'), ('٣', 'CD'), ('٢٠٢١', 'CD'), ('تقع', 'NN'), ('غرب', 'NNP'), ('قرة', 'NNP'), ('ريقي،', 'NNP'), ('حدد', 'NNP'), ('وقع', 'NNP'), ('جغراف', 'NNP'), ('إحداث', 'NNP'), ('درج', 'NNP'), ('شمل', 'NNP'), ('درج', 'NNP'), ('غربا،', 'NNP'), ('ان', 'NNP'), ('سحل', 'NNP'), ('حيط', 'NNP'), ('طلس', 'NNP'), ('بحر', 'NNP'), ('ابض', 'NNP'), ('توسط', 'NNP'), ('فصل', 'NNP'), ('بين', 'NNP'), ('ضيق', 'NNP'), ('جبل', 'NNP'), ('طارق،', 'NNP'), ('تحد', 'NNP'), ('جنب', 'NNP'), ('شرق', 'NNP'), ('دول', 'NNP'), ('جزائر،', 'NNP'), ('جنب', 'NNP'), ('جنب', 'NNP'), ('جمهور', 'NNP'), ('موريتانية،', 'NNP'), ('يشر', 'NNP'), ('ان', 'NNP'), ('ساح', 'NNP'), ('غرب', 'NNP'), ('جمل', 'NNP'), ('بلغ', 'NNP'), ('حلي', 'NNP'), ('كم١٢', 'NNP'), ('تعد', 'NNP'), ('دين', 'NNP'), ('ربط', 'NNP'), ('عصم', 'NNP'), ('غرب،', 'NNP'), ('يقم', 'NNP'), ('حلي',

with Farasa API :

In [ ]:
# Define the URL of the Farasa POS tagging API
url = 'https://farasa.qcri.org/webapi/pos/'

# API key provided by Farasa
api_key = "wSBJontyOLZkxatynQ"

# Prepare the payload data
payload = {'text': ' '.join(filtered_arabic_tokens), 'api_key': api_key}

# Send a POST request to the Farasa POS tagging API
response = requests.post(url, data=payload)

# Parse the JSON response
result = json.loads(response.text)

# Extract the list of tokens
pos = result.get('text', [])

# Print the response
print(pos)

[{'surface': 'S', 'POS': 'S', 'num': '', 'position': 'B'}, {'surface': 'ثقاف +ة', 'POS': 'NOUN+NSUFF', 'num': 'FS', 'position': 'B'}, {'surface': 'عام +ة', 'POS': 'ADJ+NSUFF', 'num': 'FS', 'position': 'B'}, {'surface': 'حول', 'POS': 'V', 'num': '', 'position': 'B'}, {'surface': 'ال+ مغرب', 'POS': 'DET+NOUN', 'num': 'MS', 'position': 'B'}, {'surface': 'تم +ت', 'POS': 'V+PRON', 'num': '', 'position': 'B'}, {'surface': 'ال+ كتاب +ة', 'POS': 'DET+NOUN+NSUFF', 'num': 'FS', 'position': 'B'}, {'surface': 'ب+', 'POS': 'PREP', 'num': '', 'position': 'B'}, {'surface': 'واسط +ة', 'POS': 'NOUN+NSUFF', 'num': 'FS', 'position': 'I'}, {'surface': 'محمد', 'POS': 'NOUN', 'num': 'MS', 'position': 'B'}, {'surface': 'مروان', 'POS': 'NOUN', 'num': 'MS', 'position': 'B'}, {'surface': 'آخر', 'POS': 'NOUN', 'num': 'MS', 'position': 'B'}, {'surface': 'تحديث', 'POS': 'NOUN', 'num': 'MS', 'position': 'B'}, {'surface': '١٨٣١', 'POS': 'NUM', 'num': 'MP', 'position': 'B'}, {'surface': '،', 'POS': 'PUNC', 'num': '',



---


## Named Entity Recognition (NER)

with NLTK :

In [ ]:
nltk.download('maxent_ne_chunker')
nltk.download('words')

In [ ]:
named_entities = ne_chunk(tagged_tokens)
print(named_entities)

(S
  ثقف/JJ
  (ORGANIZATION حول/NNP)
  غرب/NNP
  تمت/NNP
  كتب/NNP
  بسط/NNP
  حمد/NNP
  مرو/NNP
  اخر/NNP
  حدث/NNP
  ١٨٣١/CD
  ،/VBD
  ٣/CD
  ٢٠٢١/CD
  (ORGANIZATION تقع/NN)
  (ORGANIZATION غرب/NNP)
  قرة/NNP
  ريقي،/NNP
  حدد/NNP
  وقع/NNP
  جغراف/NNP
  إحداث/NNP
  درج/NNP
  شمل/NNP
  درج/NNP
  غربا،/NNP
  ان/NNP
  سحل/NNP
  حيط/NNP
  طلس/NNP
  بحر/NNP
  ابض/NNP
  توسط/NNP
  فصل/NNP
  بين/NNP
  ضيق/NNP
  جبل/NNP
  طارق،/NNP
  تحد/NNP
  جنب/NNP
  شرق/NNP
  دول/NNP
  جزائر،/NNP
  جنب/NNP
  جنب/NNP
  جمهور/NNP
  موريتانية،/NNP
  يشر/NNP
  ان/NNP
  ساح/NNP
  غرب/NNP
  جمل/NNP
  بلغ/NNP
  حلي/NNP
  كم١٢/NNP
  تعد/NNP
  دين/NNP
  ربط/NNP
  عصم/NNP
  غرب،/NNP
  يقم/NNP
  حلي/NNP
  مة،/NNP
  وفق/NNP
  إحصائ/NNP
  شهر/NNP
  عام/NNP
  م،/NNP
  وزع/NNP
  ساح/NNP
  بلغ/NNP
  كم،/NNP
  بلغ/NNP
  كثف/NNP
  سكن/NNP
  شخص/NNP
  لكل/NNP
  كيلومتر/NNP
  ربع،/NNP
  طلق/NNP
  دين/NNP
  ربط/NNP
  عدة/NNP
  قب،/NNP
  شنط/NNP
  افريقيا؛/NNP
  لأن/NNP
  شرع/NNP
  وسع/NNP
  علم/NNP
  ثري/NNP
  مميزة،/NNP
  

with Farasa API : Named entity recognition (NER) is the task of tagging entities in text with their corresponding type. Approaches typically use BIO notation, which differentiates the beginning (B) and the inside (I) of entities. O is used for non-entity tokens.

In [ ]:
# Define the URL of the Farasa NER tagging API
url = 'https://farasa.qcri.org/webapi/ner/'

# API key provided by Farasa
api_key = "wSBJontyOLZkxatynQ"

# Prepare the payload data
payload = {'text': cleaned_text, 'api_key': api_key}

# Send a POST request to the Farasa NER tagging API
data = requests.post(url, data=payload)

# Parse the JSON response
result = json.loads(data.text)

# Extract the list of tokens
pos = result.get('text', [])

# Print the response
print(pos)

['ثقافة/O', 'عامة/O', 'حول/O', 'المغرب/B-LOC', 'تمت/O', 'الكتابة/O', 'بواسطة/O', 'محمد/B-PERS', 'مروان/I-PERS', 'آخر/O', 'تحديث/O', '١٨٣١/O', '،/O', '٣/O', 'فبراير/O', '٢٠٢١/O', 'تقع/O', 'المغرب/B-LOC', 'في/O', 'شمال/O', 'قارة/O', 'أفريقي/O', '،/O', 'ويتحدد/O', 'موقعها/O', 'جغرافيا/O', 'بإحداثيات/O', 'درجة/O', 'شمالا/O', 'و/O', 'درجات/O', 'غربا/O', '،/O', 'إذ/O', 'إن/O', 'لها/O', 'سواحل/O', 'على/O', 'المحيط/B-LOC', 'الأطلسي/I-LOC', 'والبحر/B-LOC', 'الأبيض/I-LOC', 'المتوسط/I-LOC', 'الذي/O', 'يفصل/O', 'بينهما/O', 'مضيق/O', 'جبل/B-PERS', 'طارق/I-PERS', '،/O', 'كما/O', 'تحدها/O', 'من/O', 'الجانب/O', 'الشرقي/O', 'دولة/O', 'الجزائر/B-LOC', '،/O', 'ومن/O', 'الجانب/O', 'الجنوبي/O', 'الجمهورية/O', 'الموريتانية/O', '،/O', 'ويشار/O', 'إلى/O', 'أن/O', 'مساحة/O', 'المغرب/B-LOC', 'الإجمالية/O', 'تبلغ/O', 'حوالي/O', 'كم١٢/O', 'تعد/O', 'مدينة/O', 'الرباط/B-LOC', 'عاصمة/O', 'المغرب/B-LOC', '،/O', 'ويقيم/O', 'فيها/O', 'حوالي/O', 'نسمة/O', '،/O', 'وفقا/O', 'لإحصائيات/O', 'شهر/O', 'تموز/O', 'من/O', 'عام/O

# **Synthesis**

Throughout the lab, I gained valuable insights into the intricacies of web scraping, data preprocessing, and natural language processing (NLP) techniques.

Firstly, I learned how to utilize Beautiful Soup to extract data from web sources, enabling me to gather valuable information from diverse Arabic websites efficiently.

Secondly, I delved into storing the scraped data in a NoSQL database like MongoDB, both locally and on MongoDB Atlas (Cloud). This hands-on experience familiarized me with new type of databases.

Moving on to NLP, I grasped the significance of text cleaning, tokenization, and normalization in preparing textual data for analysis. Techniques such as removing stop words, punctuation, and special characters helped streamline the data and improve the quality of subsequent analyses.

Exploring stemming and lemmatization provided valuable insights into different approaches to word normalization. While stemming offered a fast and lightweight method, lemmatization preserved the semantic meaning of words, albeit at a slightly higher computational cost.

Additionally, I explored parts-of-speech (POS) tagging using both rule-based and machine learning approaches. While rule-based tagging allowed for explicit rule definitions, machine learning-based tagging leveraged annotated data to identify patterns and nuances in language usage.

Furthermore, I experimented with named entity recognition (NER) using NLTK and the Farasa API, which played a crucial role in identifying and categorizing entities such as names, organizations, and locations in Arabic text. This experience broadened my understanding of utilizing REST APIs for NLP tasks.

Overall, this lab equipped me with a comprehensive understanding of the data acquisition, preprocessing, and NLP pipeline, laying a solid foundation for future endeavors in data science and text analysis.